In [1]:
## Pick the step you want to run:
sequences = {
    '5.2': ['power-24V-on', 'power-hercules-on', 'power-hercules-off'],
    '5.3': ['to-service', 'power-on', 'wifi-mode', 'heater-test', 'deploy']
}
sequence_num, step_num = '5.2', 2
step = sequences[sequence_num][step_num]

In [2]:
from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload, WatchdogCommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr
import base64 as b64

In [3]:
settings['SAVE_FILE_PREFIX'] = 'iris_pims_5' # this the prefix on all log files. make it something unique.
load_cache()

In [4]:
## SET TO TRUE TO VIEW A FULL-LIST OF ALL COMMANDS AND TELEMETRY:
if show_commands := True:
    standards.print_overview()

Data Standards Overview: [

	Module[256]::BlockDriver

		Commands:

		Telemetry:

			0.	Channel[0]::BdCycles: uint32

		Events:

	Module[512]::RateGroupDriver

		Commands:

		Telemetry:

		Events:

	Module[768]::ActiveRateGroup-RateGroupLowFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1024]::ActiveRateGroup-RateGroupMedFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1280]::ActiveRateGroup-RateGroupHiFreq

		Commands:

		Telemetry:

			0.	Channel[0]::RgMaxTime: uint32

			1.	Channel[1]::RgCycleSlips: uint32

		Events:

			0.	Event[0]::RateGroupStarted[]

			1.	Event[1]::RateGroupCycleSlip[cycle: uint32]

	Module[1536]::CubeRoverTime

		Commands:

		Teleme

In [5]:
wired_seq_num = step_num #0x00
wireless_seq_num = 0x00

In [6]:
## Pre-prepared commands list:

source = DataSource.GENERATED

prepared_commands = {
    'to-service': ( # Tell the Watchdog to switch into service mode
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_SwitchToServiceMode',
        dict(confirm='CONFIRM_SERVICE'),
        DataPathway.WIRED
    ),
    'power-on': ( # Turn Everything On
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_PrepareForDeployment',
        dict(confirm='CONFIRM_PREP'),
        DataPathway.WIRED
    ),
    'wifi-mode': ( # Turn Everything On
        DataPathway.WIRED, # intentionally wired to test the wd to herc command path
        Magic.COMMAND, # "normal" command is for Hercules
        'GroundInterface_SetPrimaryInterface',
        dict(primary_interface='WF_121'),
        DataPathway.WIRELESS
    ),
    'deploy': (
        DataPathway.WIRED,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'deploy-wd-only': ( # special command to tell only WD to release its interlock (in case Herc-WD comms are broken)
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'deploy-herc-only': ( # special command to tell only Herc to release its interlock (in case Herc-WD comms are broken)
        DataPathway.WIRELESS,
        Magic.COMMAND, # "normal" command is for Hercules
        'WatchDogInterface_DisengageFromLander',
        dict(confirm='CONFIRM_DEPLOY'),
        DataPathway.WIRELESS
    ),
    'power-24V-on': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='EN_24_ON'),
        DataPathway.WIRED
    ),
    'power-hercules-on': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HERCULES_POWER_ON'),
        DataPathway.WIRED
    ),
    'power-hercules-off': (
        DataPathway.WIRED,
        Magic.WATCHDOG_COMMAND,
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='HERCULES_POWER_OFF'),
        DataPathway.WIRED
    ),
    'heater-test': (
        DataPathway.WIRED,
        Magic.COMMAND, # intentionally telling the WD to tell Herc to tell the WD to enable heater control (same path as deployment command but a quick pretest)
        'WatchDogInterface_ResetSpecific',
        dict(reset_value='ENABLE_HEATER_CONTROL'),
        DataPathway.WIRELESS
    ),
}
pathway, magic, command_name, kwargs, telem_pathway = prepared_commands[step]
print(pathway, magic, command_name, kwargs, telem_pathway)

DataPathway.WIRED Magic.WATCHDOG_COMMAND WatchDogInterface_ResetSpecific {'reset_value': 'HERCULES_POWER_OFF'} DataPathway.WIRED


In [7]:
## Build Command:

if pathway == DataPathway.WIRED:
    wired_seq_num += 1
    seq_num = wired_seq_num
elif pathway == DataPathway.WIRELESS:
    wireless_seq_num += 1
    seq_num = wireless_seq_num

command_payload_type = {
    Magic.WATCHDOG_COMMAND: WatchdogCommandPayload,
    Magic.COMMAND: CommandPayload
}[magic]

module, command = standards.global_command_lookup(command_name)
payloads = EnhancedPayloadCollection(
    CommandPayload=[
        command_payload_type(
            pathway=pathway,
            source=source,
            magic=magic,
            module_id=module.ID,
            command_id=command.ID,
            args=kwargs
        )
    ],
    TelemetryPayload=[],
    EventPayload=[],
    FileBlockPayload=[]
)
packet = IrisCommonPacket(
    seq_num = seq_num,
    payloads = payloads
).encode()
print(f"{pathway} Command #{seq_num}:\t {hexstr(packet)}")

DataPathway.WIRED Command #3:	 03 07 00 35 ee ff 00 c0 00 10 03


In [8]:
yamcs_payload = {
    'tc-binary': hexstr(packet).replace(' ','')
}
yamcs_payload

{'tc-binary': '03070035eeff00c0001003'}

In [9]:
yamcs_name_root = command_name
# yamcs_name_root = 'WatchDogInterface_ResetSpecific'

# import requests
# url = '192.168.10.2:8090/api/processors/Astrobotic-M1/realtime/commands/Astrobotic-M1/WatchDogInterface_SwitchToServiceMode'
# auth = ('iris', 'justanexample')
# headers = {'Content-Type': 'application/json'}
HOSTNAME = "192.168.10.105"
PORT = "8090"
URI = "{0}:{1}".format(HOSTNAME, PORT)

YAMCS_INSTANCE = 'Astrobotic-M1'
YAMCS_PROCESSOR = 'realtime'

from yamcs.client import YamcsClient
from yamcs.core.auth import Credentials as YamcsCredentials

# Sign in with username and password defined at top of file
credentials = YamcsCredentials(username='iris', password='justanexample')

# Create a yamcs client that connects with our server.
# Currently connects to localhost but will change later
client = YamcsClient(URI, credentials=credentials)

# Send a command:
yamcs_cmd_name = f"/{YAMCS_INSTANCE}/{yamcs_name_root}{pathway.yamcs_suffix}"
processor = client.get_processor(instance=YAMCS_INSTANCE, processor=YAMCS_PROCESSOR)
status = processor.issue_command(yamcs_cmd_name, args=yamcs_payload)
print("Command status: ", status.__dict__)

In [ ]:
# Stream YAMCS telemetry:
payload_name = 'iris'
parameter = f"/Astrobotic-M1/{payload_name}_payload-tm-rs422"
# parameter = f"/Astrobotic-M1/{payload_name}-payload-tm-wlan"
# parameter = f"/Astrobotic-M1/{payload_name}-power" # is pwr on
# parameter = f"/Astrobotic-M1/{payload_name}-current" # [Amps]
# parameter = f"/Astrobotic-M1/{payload_name}-downlink" # [bps]

processor = client.get_processor(instance=YAMCS_INSTANCE, processor=YAMCS_PROCESSOR)

subscription = processor.create_parameter_subscription([parameter])

while(1):
    sleep(1)
    param_val = subscription.get_value(parameter)
    print(f"Latest value: {param_val}")
    # Extract the parameter data:
    bin_val = param_val.raw_value
    # bin_val = getattr(getattr(param_val, 'rawValue'), 'binaryValue')
    # bin_val = b64.b64decode(bin_value.encode('ascii')) # decode b64 string to byte array
    # Decode packet:
    packet = parse_packet(bin_val)
    print(f"Contents: {packet}")

In [ ]:
# Decode manual packet:
"""
0AEC00
2021-05-24T13:28:52.000Z	FFFF0F0000000000000000F401FD0C3C0034210401013421
2021-05-24T13:28:59.011Z	FFFF0F0000000000000000F401FD0C3C0034210401010000
2021-05-24T13:29:07.006Z	FFFF0F0000000000000000F401FD0C3C0034210401010000
"""
bin_val = bytes.fromhex('00890254FF1000C0000F00000000FFFFFF1000C0010F00000000FEFFFF1000C0020F00000000A100FF1000C0030F000000006300FF1000C0040F000000000D00FF1000C0050F000000004401FF1000C000050000000000000000FF1000C0000402000000D0070000FF1000C000090200000000FF1000C002090200000000000000FF1000C003090200000000000000FF1000C0000324000000A08C0000FF1000C0000FF2000000FCFFFF1000C0010FF2000000FCFFFF1000C0020FF2000000A300FF1000C0030FF20000004A00FF1000C0040FF2000000F3FFFF1000C0050FF20000005101FF1000C0000525000000E8030000FF1000C0000FE5010000FAFFFF1000C0010FE5010000FDFFFF1000C0020FE5010000A800FF1000C0030FE50100005A00FF1000C0040FE50100000E00FF1000C0050FE50100005D01FF1000C00009F500000000FF1000C00209F500000000000000FF1000C00309F500000000000000FF1000C00009E701000000FF1000C00209E701000000000000FF1000C00309E701000000000000FF1000C0000FD7020000FAFFFF1000C0010FD7020000FEFFFF1000C0020FD7020000A300FF1000C0030FD70200005000FF1000C0040FD70200001800FF1000C0050FD70200007801FF1000C0000433020000B80B0000FF1000C0000FCA030000FDFFFF1000C0010FCA030000FDFFFF1000C0020FCA030000A100FF1000C0030FCA0300003300FF1000C0040FCA030000EAFFFF1000C0050FCA0300001501FF1000C00009DA02000000FF1000C00209DA02000000000000FF1000C00309DA02000000000000FF1000C00009CC03000000FF1000C00209CC03000000000000FF1000C00309CC03000000000000FF1000C0000FA4080000FEFFFF1000C0010FA4080000FCFF')
packet = parse_packet(bin_val)
print(f"Contents: {packet}")


In [ ]:
# connect_serial(device = '/dev/tty.usbserial-A7035PDL')
# send_command_wd_serial('WatchDogInterface_ResetSpecific', reset_value='RESET_HERCULES')

'Data: 45 00 00 27 00 01 00 00 40 11 5e 27 de ad be ef 7f 00 00 01 00 35 1f 90 00 13 d2 8d 00 07 00 00 ee ff 00 c0 00 10 01.'

In [ ]:
# stream_data_ip_udp_serial()

le: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat: OFF, Ctrl: ON] 	260.9°K -> 273.6°K +- 0.92K° 	Kp = 500 @ Duty Cycle: 0/65535

RS_MISSION:	[Heat